In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# For baseline / factorized model

# Load model
# Hook given layer
# Loop over data 
    # forward pass 1 obs -> stores 
    # Store feature embedding of 

In [3]:
# https://forums.fast.ai/t/pytorch-best-way-to-get-at-intermediate-layers-in-vgg-and-resnet/5707/3

# outputs= []
# def hook(module, input, output):
#     outputs.append(output)

# baseline_model.layer1[0].conv1.register_forward_hook(hook)

# batch = next(iter(train_loader))
# img, label = batch

# out = baseline_model(img.cuda())

In [4]:
import random
from pathlib import Path
import numpy as np
import torch

seed = 0
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

In [5]:
path = Path('/bigdata/f_mnist/logs/baselines/1644945876/rn18_18_dNone_256_adam_l0.001_g0.1_sTrue')

layer = 28
logdir = path / f'features_layer{layer}_seed{seed}'
logdir.mkdir(parents=True, exist_ok=True)


baseline_path = Path('/bigdata/f_mnist/logs/baselines/1644945876/rn18_18_dNone_256_adam_l0.001_g0.1_sTrue/cnn_best.pth')
baseline_model = torch.load(path / "cnn_best.pth")

In [6]:
from tddl.data.loaders import fmnist_stratified_loaders

data_dir = Path("/bigdata/f_mnist")
batch_size = 1
data_workers = 8

train_loader, valid_loader, test_loader = fmnist_stratified_loaders(
    path=data_dir,
    batch_size=batch_size,
    data_workers=data_workers,
    valid_size=5000,
    random_transform_training=False,
)

In [7]:
outputs= []
def hook(module, input, output):
    outputs.append(output)

def hook_network(
    model, layers=[], exclude=[], verbose=False, 
    return_error=False, 
    **kwargs,
):
    """
    Usage: factorize_network(model, layers=[6])
    """
    i = -1
    def nested_children(m: torch.nn.Module, **kwargs):
        """
        layers: List of either
            layer numbers (numbered according to number_layers(model))
            layer names
            module types
        """
        nonlocal i
        children = dict(m.named_children())
        output = {}
        error = None

        i+=1
        
        if children == {}:
            return m
        else:
        # look for children from children... to the last child!
            for name, child in children.items():
                if verbose:
                    print(i, name, type(child))
                if name in exclude:
                    i+=1
                    continue
                # if type(child) == torch.nn.modules.conv.Conv2d and i in layers:
                if i in layers or name in layers or type(child) in layers:
                    # if return_error:
                    #     layer, error = factorize_layer(child, **kwargs)
                    #     if verbose:
                    #         print(error)
                    # else:
                    #     layer = 
                    m._modules[name].register_forward_hook(hook)
                try:
                    # if verbose and return_error:
                    #     print((i, error))
                    output[name] = (i, nested_children(child, **kwargs) )
                except TypeError:
                    output[name] = (i, nested_children(child, **kwargs) )
        return output #, errors
    out = nested_children(model, **kwargs)

    if return_error:
        return out

In [8]:
baseline_model = torch.load(baseline_path)
hook_network(baseline_model, layers=[layer])

In [9]:
from tqdm import tqdm

outputs = []
with torch.no_grad():
    for i, (img, label) in enumerate(tqdm(train_loader)):
        out = baseline_model(img.cuda())
        features = outputs.pop()
        torch.save(features, logdir / f"features_i{i}.pth")

55000it [02:57, 309.90it/s]


In [12]:
# test if hooking features works for factorized model

layer = 15 # from config file
fact_model = torch.load("/bigdata/f_mnist/logs/decomposed_adam/1645014717/rn18-lr-[15]-cp-0.5-dTrue-iNone_bn_256_sgd_l1e-05_g0.0_sTrue/fact_model_best.pth")
hook_network(fact_model, layers=[layer])
fact_model(img.cuda())
outputs[0]

tensor([[[[ 5.3680e-01,  5.4497e-01,  9.0979e-01,  ..., -6.5729e-02,
            1.2381e+00,  1.8000e+00],
          [ 1.2621e+00,  4.4105e-01,  9.1233e-01,  ..., -3.6674e+00,
           -2.8421e+00, -1.4289e+00],
          [ 5.1122e-01,  1.4222e-01,  4.4297e-01,  ..., -6.6928e-01,
           -3.3594e+00, -3.5585e+00],
          ...,
          [ 3.5719e+00,  2.4761e+00, -4.2372e-01,  ..., -7.1231e-01,
           -2.6409e+00,  2.2242e-01],
          [ 6.3407e+00,  7.1744e+00,  7.1558e+00,  ...,  6.6040e+00,
            5.1314e+00,  4.7735e+00],
          [ 4.8656e-01,  1.6399e-01,  1.8810e+00,  ...,  7.6749e-01,
            4.5924e-01,  6.5591e-01]],

         [[ 1.9712e-01,  7.3736e-01,  5.7075e-01,  ..., -2.7825e+00,
           -3.8005e+00, -2.9572e+00],
          [ 7.3113e-02,  3.6040e-01,  3.0848e-01,  ..., -9.2888e-02,
           -9.5780e-01,  4.9804e-01],
          [-9.7015e-01, -1.3747e+00, -1.2999e+00,  ...,  1.8417e+00,
            1.2893e+00,  2.9309e+00],
          ...,
     